In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

In [ ]:
# Parameters and Settings

# Parameters for data split
WINDOW = 21    # rolling window size to use as predictors
DATE_COL = 'Date'
ID_COL = 'PERMNO'
TARGET_COL = 'excess_return'

# File path for the cleaned and filtered data file
current_directory = os.getcwd()
clean_filtered_data_path = os.path.join(current_directory, 'Data', 'clean_filtered_data.csv')

# File path to save prediction results
results_path = os.path.join(current_directory, 'Results', f'models_results{WINDOW:.0f}.csv')

# File path to save best model parameters
best_parameters_path = os.path.join(current_directory, 'Results', f'models_best_parameters{WINDOW:.0f}.csv')

# Estimation (in sample) period dates
in_sample_start_date = pd.to_datetime("2000-01-01")
in_sample_end_date = pd.to_datetime("2015-12-31")

# Out-of-sample period dates
out_sample_start_date = pd.to_datetime("2016-01-01")
out_sample_end_date = pd.to_datetime("2024-12-31")

### Step 1: Load and Process Data

In [ ]:
# Load the cleaned and filtered data files for in sample and out of sample periods into a pandas DataFrames (output from ETL step)
df = pd.read_csv(clean_filtered_data_path)

# Ensure the date columns are in datetime format
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

df = df[[ID_COL, DATE_COL, TARGET_COL]].dropna()
df = df.sort_values([ID_COL, DATE_COL]).reset_index(drop=True)

df.info()

In [ ]:
# Check number of unique stocks
stocks_permno = df["PERMNO"].unique().tolist()
print(f"Number of unique stocks: {len(stocks_permno)}")

In [ ]:
# Create rolling window for predictors
for lag in range(1, WINDOW+1):
    df[f'lag_{lag}'] = df.groupby(ID_COL)[TARGET_COL].shift(lag)
df = df.dropna(subset=[f'lag_{lag}' for lag in range(1, WINDOW+1)]).reset_index(drop=True)

df.info()

In [ ]:
# Spit data into estimation (in-sample) and out-of-sample data
df_train = df[(df[DATE_COL] >= in_sample_start_date) & (df[DATE_COL] <= in_sample_end_date)].copy().reset_index(drop=True)
df_test = df[(df[DATE_COL] >= out_sample_start_date) & (df[DATE_COL] <= out_sample_end_date)].copy().reset_index(drop=True)

print(df_train.info())
print(df_test.info())

In [ ]:
X_train = df_train[[f'lag_{lag}' for lag in range(1, WINDOW+1)]].values
y_train = df_train[TARGET_COL].values

X_test = df_test[[f'lag_{lag}' for lag in range(1, WINDOW+1)]].values
y_test = df_test[TARGET_COL].values

results = df_test[[ID_COL, DATE_COL, TARGET_COL]]

### Step 2: Training Models and Hyperparameters Tuning

In [ ]:
# Create a Function to Calculate Predictive-R2 Used in the Finance Literature
def r2(y_true, y_pred):
    return 1-(((y_true-y_pred)**2).sum()/(y_true**2).sum())

In [ ]:
# Linear Models

# Ordinary Least Square
ols = LinearRegression(n_jobs=-1)
ols.fit(X_train, y_train)
y_ols = ols.predict(X_test)

# Penalized Linear Regression
r2_lasso_best = -1000
r2_ridge_best = -1000
r2_enet_best = -1000

for alpha in np.logspace(-10,-5,11):
    # Lasso
    lasso = Lasso(alpha=alpha, max_iter=2000, random_state=73, selection='random')
    lasso.fit(X_train, y_train)
    pred_lasso = lasso.predict(X_test)
    r2_lasso_temp = r2(y_test, pred_lasso)
    if r2_lasso_temp > r2_lasso_best:
        y_lasso = pred_lasso
        alpha_lasso = alpha
        r2_lasso_best = r2_lasso_temp

    # Elastic Net
    for l1_r in [0.25,0.5,0.75]:
        enet = ElasticNet(alpha=alpha, l1_ratio=l1_r, max_iter=2000, random_state=73, selection='random')
        enet.fit(X_train, y_train)
        pred_enet = enet.predict(X_test)
        r2_enet_temp = r2(y_test, pred_enet)
        if r2_enet_temp > r2_enet_best:
            y_enet = pred_enet
            alpha_enet = alpha
            l1_r_enet = l1_r
            r2_enet_best = r2_enet_temp

for alpha in np.logspace(-5,0,11):
    # Ridge
    ridge = Ridge(alpha=alpha, max_iter=2000, random_state=73)
    ridge.fit(X_train, y_train)
    pred_ridge = ridge.predict(X_test)
    r2_ridge_temp = r2(y_test, pred_ridge)
    if r2_ridge_temp > r2_ridge_best:
        y_ridge = pred_ridge
        alpha_ridge = alpha
        r2_ridge_best = r2_ridge_temp


results["y_ols"] = y_ols
results["y_lasso"] = y_lasso
results["y_enet"] = y_enet
results["y_ridge"] = y_ridge

In [ ]:
# Non-Linear Models

# Random Forest Regression
r2_rf_best = -1000

for leaf_size in [2, 5, 10, 20, 50]:
    rf = RandomForestRegressor(n_estimators=300, max_depth=10, min_samples_leaf=leaf_size, n_jobs=-1, random_state=73)
    rf.fit(X_train, y_train)
    pred_rf = rf.predict(X_test)
    r2_rf_temp = r2(y_test, pred_rf)
    if r2_rf_temp > r2_rf_best:
        y_rf = pred_rf
        leaf_size_rf = leaf_size
        r2_rf_best = r2_rf_temp

# XGBoost Regression
r2_xgb_best = -1000

for lr in [0.01, 0.1]:
    xgb = XGBRegressor(n_estimators=1000, max_depth=2, learning_rate=lr, random_state=73, device="cuda" if torch.cuda.is_available() else "cpu")
    xgb.fit(X_train, y_train)
    pred_xgb = xgb.predict(X_test)
    r2_xgb_temp = r2(y_test, pred_xgb)
    if r2_xgb_temp > r2_xgb_best:
        y_xgb = pred_xgb
        lr_xgb = lr
        r2_xgb_best = r2_xgb_temp


results["y_rf"] = y_rf
results["y_xgb"] = y_xgb

In [ ]:
# Neural Networks

# Model 1 (1 Layer)
r2_nn1_best = -1000

for alpha in [0.00001, 0.0001, 0.001]:
    for lr in [0.001, 0.01]:
        nn1 = MLPRegressor(hidden_layer_sizes=(64,), alpha=alpha, batch_size=500, learning_rate_init=lr, max_iter=100, random_state=73, early_stopping=True)
        nn1.fit(X_train, y_train)
        pred_nn1 = nn1.predict(X_test)
        r2_nn1_temp = r2(y_test, pred_nn1)
        if r2_nn1_temp > r2_nn1_best:
            y_nn1 = pred_nn1
            alpha_nn1 = alpha
            lr_nn1 = lr
            r2_nn1_best = r2_nn1_temp

# Model 2 (2 Layers)
r2_nn2_best = -1000

for alpha in [0.00001, 0.0001, 0.001]:
    for lr in [0.001, 0.01]:
        nn2 = MLPRegressor(hidden_layer_sizes=(64,32,), alpha=alpha, batch_size=500, learning_rate_init=lr, max_iter=100, random_state=73, early_stopping=True)
        nn2.fit(X_train, y_train)
        pred_nn2 = nn2.predict(X_test)
        r2_nn2_temp = r2(y_test, pred_nn2)
        if r2_nn2_temp > r2_nn2_best:
            y_nn2 = pred_nn2
            alpha_nn2 = alpha
            lr_nn2 = lr
            r2_nn2_best = r2_nn2_temp

# Model 3 (3 Layers)
r2_nn3_best = -1000

for alpha in [0.00001, 0.0001, 0.001]:
    for lr in [0.001, 0.01]:
        nn3 = MLPRegressor(hidden_layer_sizes=(64,32,16,), alpha=alpha, batch_size=500, learning_rate_init=lr, max_iter=100, random_state=73, early_stopping=True)
        nn3.fit(X_train, y_train)
        pred_nn3 = nn3.predict(X_test)
        r2_nn3_temp = r2(y_test, pred_nn3)
        if r2_nn3_temp > r2_nn3_best:
            y_nn3 = pred_nn3
            alpha_nn3 = alpha
            lr_nn3 = lr
            r2_nn3_best = r2_nn3_temp

# Model 4 (4 Layers)
r2_nn4_best = -1000

for alpha in [0.00001, 0.0001, 0.001]:
    for lr in [0.001, 0.01]:
        nn4 = MLPRegressor(hidden_layer_sizes=(64,32,16,8,), alpha=alpha, batch_size=500, learning_rate_init=lr, max_iter=100, random_state=73, early_stopping=True)
        nn4.fit(X_train, y_train)
        pred_nn4 = nn4.predict(X_test)
        r2_nn4_temp = r2(y_test, pred_nn4)
        if r2_nn4_temp > r2_nn4_best:
            y_nn4 = pred_nn4
            alpha_nn4 = alpha
            lr_nn4 = lr
            r2_nn4_best = r2_nn4_temp

# Model 5 (5 Layers)
r2_nn5_best = -1000

for alpha in [0.00001, 0.0001, 0.001]:
    for lr in [0.001, 0.01]:
        nn5 = MLPRegressor(hidden_layer_sizes=(64,32,16,8,4,), alpha=alpha, batch_size=500, learning_rate_init=lr, max_iter=100, random_state=73, early_stopping=True)
        nn5.fit(X_train, y_train)
        pred_nn5 = nn5.predict(X_test)
        r2_nn5_temp = r2(y_test, pred_nn5)
        if r2_nn5_temp > r2_nn5_best:
            y_nn5 = pred_nn5
            alpha_nn5 = alpha
            lr_nn5 = lr
            r2_nn5_best = r2_nn5_temp


results["y_nn1"] = y_nn1
results["y_nn2"] = y_nn2
results["y_nn3"] = y_nn3
results["y_nn4"] = y_nn4
results["y_nn5"] = y_nn5

### Step 3: Evaluate Statistical Performance of Models

In [ ]:
# Evaluate models

# Linear Models
r2_ols = r2(y_test, y_ols)
mse_ols = mean_squared_error(y_test, y_ols)
mae_ols = mean_absolute_error(y_test, y_ols)
da_ols = (np.sign(y_test) == np.sign(y_ols)).mean()

r2_lasso = r2_lasso_best
mse_lasso = mean_squared_error(y_test, y_lasso)
mae_lasso = mean_absolute_error(y_test, y_lasso)
da_lasso = (np.sign(y_test) == np.sign(y_lasso)).mean()

r2_enet = r2_enet_best
mse_enet = mean_squared_error(y_test, y_enet)
mae_enet = mean_absolute_error(y_test, y_enet)
da_enet = (np.sign(y_test) == np.sign(y_enet)).mean()

r2_ridge = r2_ridge_best
mse_ridge = mean_squared_error(y_test, y_ridge)
mae_ridge = mean_absolute_error(y_test, y_ridge)
da_ridge = (np.sign(y_test) == np.sign(y_ridge)).mean()

# Non-Linear Models
r2_rf = r2_rf_best
mse_rf = mean_squared_error(y_test, y_rf)
mae_rf = mean_absolute_error(y_test, y_rf)
da_rf = (np.sign(y_test) == np.sign(y_rf)).mean()

r2_xgb = r2_xgb_best
mse_xgb = mean_squared_error(y_test, y_xgb)
mae_xgb = mean_absolute_error(y_test, y_xgb)
da_xgb = (np.sign(y_test) == np.sign(y_xgb)).mean()

# Neural Networks
r2_nn1 = r2_nn1_best
mse_nn1 = mean_squared_error(y_test, y_nn1)
mae_nn1 = mean_absolute_error(y_test, y_nn1)
da_nn1 = (np.sign(y_test) == np.sign(y_nn1)).mean()

r2_nn2 = r2_nn2_best
mse_nn2 = mean_squared_error(y_test, y_nn2)
mae_nn2 = mean_absolute_error(y_test, y_nn2)
da_nn2 = (np.sign(y_test) == np.sign(y_nn2)).mean()

r2_nn3 = r2_nn3_best
mse_nn3 = mean_squared_error(y_test, y_nn3)
mae_nn3 = mean_absolute_error(y_test, y_nn3)
da_nn3 = (np.sign(y_test) == np.sign(y_nn3)).mean()

r2_nn4 = r2_nn4_best
mse_nn4 = mean_squared_error(y_test, y_nn4)
mae_nn4 = mean_absolute_error(y_test, y_nn4)
da_nn4 = (np.sign(y_test) == np.sign(y_nn4)).mean()

r2_nn5 = r2_nn5_best
mse_nn5 = mean_squared_error(y_test, y_nn5)
mae_nn5 = mean_absolute_error(y_test, y_nn5)
da_nn5 = (np.sign(y_test) == np.sign(y_nn5)).mean()

In [ ]:
# Collate Results

results_matrix = [
    {
        "Model": "OLS",
        "R-squared": r2_ols,
        "MSE": mse_ols,
        "MAE": mae_ols,
        "Direction Accuracy": da_ols
    },
    {
        "Model": "Lasso",
        "R-squared": r2_lasso,
        "MSE": mse_lasso,
        "MAE": mae_lasso,
        "Direction Accuracy": da_lasso
    },
    {
        "Model": "Ridge",
        "R-squared": r2_ridge,
        "MSE": mse_ridge,
        "MAE": mae_ridge,
        "Direction Accuracy": da_ridge
    },
    {
        "Model": "Elastic Net",
        "R-squared": r2_enet,
        "MSE": mse_enet,
        "MAE": mae_enet,
        "Direction Accuracy": da_enet
    },
    {
        "Model": "Random Forest",
        "R-squared": r2_rf,
        "MSE": mse_rf,
        "MAE": mae_rf,
        "Direction Accuracy": da_rf
    },
    {
        "Model": "XGBoost",
        "R-squared": r2_xgb,
        "MSE": mse_xgb,
        "MAE": mae_xgb,
        "Direction Accuracy": da_xgb
    },
    {
        "Model": "NN 1",
        "R-squared": r2_nn1,
        "MSE": mse_nn1,
        "MAE": mae_nn1,
        "Direction Accuracy": da_nn1
    },
    {
        "Model": "NN 2",
        "R-squared": r2_nn2,
        "MSE": mse_nn2,
        "MAE": mae_nn2,
        "Direction Accuracy": da_nn2
    },
    {
        "Model": "NN 3",
        "R-squared": r2_nn3,
        "MSE": mse_nn3,
        "MAE": mae_nn3,
        "Direction Accuracy": da_nn3
    },
    {
        "Model": "NN 4",
        "R-squared": r2_nn4,
        "MSE": mse_nn4,
        "MAE": mae_nn4,
        "Direction Accuracy": da_nn4
    },
    {
        "Model": "NN 5",
        "R-squared": r2_nn5,
        "MSE": mse_nn5,
        "MAE": mae_nn5,
        "Direction Accuracy": da_nn5
    }
]

results_matrix_df = pd.DataFrame(results_matrix)
results_matrix_df

In [ ]:
# Best Paramters

best_parameters_matrix = [
    {
        "Model": "OLS",
        "Hidden Layers": "-",
        "Parameters": "-"
    },
    {
        "Model": "Lasso",
        "Hidden Layers": "-",
        "Parameters": f"Alpha = {alpha_lasso}"
    },
    {
        "Model": "Ridge",
        "Hidden Layers": "-",
        "Parameters": f"Alpha = {alpha_ridge}"
    },
    {
        "Model": "Elastic Net",
        "Hidden Layers": "-",
        "Parameters": f"Alpha = {alpha_enet}; L1 Ratio = {l1_r_enet}"
    },
    {
        "Model": "Random Forest",
        "Hidden Layers": "-",
        "Parameters": f"Minimum Sample per Leaf = {leaf_size_rf}"
    },
    {
        "Model": "XGBoost",
        "Hidden Layers": "-",
        "Parameters": f"Learning Rate = {lr_xgb}"
    },
    {
        "Model": "NN 1",
        "Hidden Layers": "32",
        "Parameters": f"Alpha = {alpha_nn1}; Learning Rate = {lr_nn1}"
    },
    {
        "Model": "NN 2",
        "Hidden Layers": "32, 16",
        "Parameters": f"Alpha = {alpha_nn2}; Learning Rate = {lr_nn2}"
    },
    {
        "Model": "NN 3",
        "Hidden Layers": "32, 16, 8",
        "Parameters": f"Alpha = {alpha_nn3}; Learning Rate = {lr_nn3}"
    },
    {
        "Model": "NN 4",
        "Hidden Layers": "32, 16, 8, 4",
        "Parameters": f"Alpha = {alpha_nn4}; Learning Rate = {lr_nn4}"
    },
    {
        "Model": "NN 5",
        "Hidden Layers": "32, 16, 8, 4, 2",
        "Parameters": f"Alpha = {alpha_nn5}; Learning Rate = {lr_nn5}"
    }
]

best_parameters_matrix_df = pd.DataFrame(best_parameters_matrix)
best_parameters_matrix_df

##### Save Results

In [ ]:
# Save Prediction Results and Best Model Parameters
results.to_csv(results_path, index=False)
best_parameters_matrix_df.to_csv(best_parameters_path, index=False)